In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('labels.csv')

image_paths = df['filename'].tolist()
labels = df['label'].tolist()

In [ ]:
len(image_paths)


In [ ]:
len(labels)

In [5]:
from collections import Counter

In [ ]:
Counter(labels)

In [7]:
df = pd.DataFrame({
    'image_path': image_paths,
    'label': labels
})

In [ ]:
len(df)

In [ ]:
df.sample(5)


In [27]:
import cohere
import os
from dotenv import load_dotenv, find_dotenv
from PIL import Image
from io import BytesIO
import base64
from tqdm import tqdm

In [28]:
_= load_dotenv(find_dotenv())

In [ ]:
api_key = os.getenv("COHERE_API_KEY")
model_id = os.getenv("COHERE_EMBED_MODEL_ID")
model_id

In [33]:
def image_to_base64_data_url(image_path):
    with Image.open(image_path) as img:
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
    data_url = f"data:image/jpeg;base64,{img_base64}"
    return data_url

In [34]:
co = cohere.Client(api_key=api_key)

In [ ]:
import pandas as pd

# CSVファイルのヘッダーを初期化（最初の1回のみ）
pd.DataFrame(columns=['image_path', 'label', 'embedding']).to_csv('image_embeddings.csv', index=False)

for i, image_path in enumerate(tqdm(image_paths)):
    data_url = image_to_base64_data_url(image_path)
    ret = co.embed(
        input_type="image",
        images=[data_url],
        model=model_id,
        embedding_types=["float"],                
    )
    # 1件分のデータをDataFrameとして作成
    embedding_df = pd.DataFrame([{
        'image_path': str(image_path),
        'label': labels[i],
        'embedding': ret.embeddings.float[0]
    }])
    
    # mode='a'（append）とheader=Falseで追記モードで保存
    embedding_df.to_csv('image_embeddings.csv', mode='a', header=False, index=False)

